In [1]:
import sys
paths = [
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/util",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/config",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common/ds",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_intrinio",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common/web",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_portfolio",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/test/selenium/api",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/test/selenium",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/estypes",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/esmappings",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/esindexes",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/utils",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch_ingestion/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch_ingestion",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/util",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_quandl",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_snp",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/public",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_barchart",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_redis",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/models",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_twitter",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_edgar",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_fred",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_oecd",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_websocket",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/util/serverless/elasticsearch/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/symbol_mapping",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/sbt_elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_cryptocurrency",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/user",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_cbonds",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_brokers"
]

In [2]:
sys.path += paths
# sys.path.append("C:\\Users\\tbui\\Documents\\Stansberry-Terminal-MkI\\services\\screener_sbanalysis_api\\")

In [10]:

from pandas.io import sql as psql

In [11]:
from lib.connected_db import PgsqlExecutor

In [12]:
research_db = {
        "host": "dev-df-postgresql.csodrrohkuas.us-east-1.rds.amazonaws.com",
        "port": 5432,
        "user": "df_user",
        "credentials": "Piq914sT",
        "database": "datafactory"
      }
snpsource = {
        "host": "snp-instance-1-us-east-1b.csodrrohkuas.us-east-1.rds.amazonaws.com",
        "port": 5432,
        "user": "snpx_read",
        "credentials": "Nsp563Xz",
        "database": "xpressfeed"
      }

In [13]:
db = PgsqlExecutor(snpsource)
model_db = PgsqlExecutor(research_db)

In [14]:
item_dict = {}
sql = """

select tradingitemid, tickersymbol from v2mv_sbt_company where  
is_primary_exchange = true and exchangesymbol not ilike 'mutualfund'
    and securityname ilike 'common%stock' 
             and exchangecountry = 'USA'
"""

data = db.fetch_all_rows(sql)
for row in data.query_data:
    item_dict[row.tickersymbol.upper()] = row

In [15]:
def get_current_price(symbol):
    sql = """
    SELECT  priceclose
	    FROM public.v_price_timeseries 
	        WHERE tradingitemid = %s 
	        AND pricingdate >= now()::Date -3
            
            order by pricingdate desc limit 1
    
    """
    rv = db.fetch_one_row(sql, args=[item_dict[symbol.upper()].tradingitemid])
    if rv.row_count > 0:
        return rv.query_data.priceclose

In [16]:
get_current_price('aapl')

351.73

In [17]:
from datetime import timedelta, datetime

def get_min_max_current_price(symbol, pricing_date):
    sql = """
    SELECT  priceclose, extract (epoch from pricingdate) timeseries, pricingdate
	    FROM public.v_price_timeseries 
	        WHERE tradingitemid = %s 
	        AND pricingdate between %s and %s order by timeseries;
    """
    now = pricing_date
    next_year = now + timedelta(days=365)
    tradingitemid = item_dict.get(symbol)
    
    if not tradingitemid:
        return
    
    data = db.get_dataframe(sql, (tradingitemid[0], now, next_year,))
    end_pricing_date = data['pricingdate'].iloc[-1]
    year_end_price = float(data['priceclose'].iloc[-1])
    min_price = float(data['priceclose'].min())
    max_price = float(data['priceclose'].max())
    return [min_price, max_price, year_end_price, end_pricing_date]

In [18]:
start = datetime(2018, 12, 31)
get_min_max_current_price("AAPL", start)

[142.19, 293.65, 293.65, Timestamp('2019-12-31 00:00:00')]

In [19]:
from mapper.predict_logic_mapper import MapperPredictLogic
from mapper.dcf_model_rank_mapper import MapperDcfRanking

In [20]:
model_db = PgsqlExecutor(research_db)

In [21]:
dcf_mapper = MapperDcfRanking(model_db)
logic_mapper = MapperPredictLogic(model_db)

In [22]:
# """
# drop table model.dcf_values_rank;

# CREATE TABLE model.dcf_values_rank(
#     id serial NOT NULL PRIMARY KEY,
#     symbol character varying(10) NOT NULL,
#     exchangesymbol character varying(50),
#     tradingitemid integer,
#     price numeric,
#     pricing_date timestamp without time zone,
#     average_growth_rate numeric,
#     adjusted_growth numeric,
#     is_yearly_consistent_rate integer,
#     dcf numeric,
#     market_dcf numeric,
#     adjusted_dcf numeric,
#     adjusted_dcf_to_price numeric,
#     year_min_price numeric,
#     year_max_price numeric,
#     year_end_price numeric,
#     end_pricing_date timestamp without time zone,,
#     dcf_ai_score numeric,
#     fiscal_year integer,

#     created timestamp without time zone DEFAULT now()

# );

#     create index ind_dcf_values_rank_symbol_idx on model.dcf_values_rank(symbol);
# 	create index ind_dcf_values_rank_symbol_fiscal_year_idx on model.dcf_values_rank(symbol, fiscal_year);
#     GRANT INSERT, SELECT, UPDATE, DELETE ON TABLE model.dcf_values_rank TO df_user;
#     GRANT all on sequence model.dcf_values_rank_id_seq to df_user;

# """


In [26]:
year_sql = """

select 
    symbol, exchangesymbol, tradingitemid,
    current_price, market_growth_rate, current_growth_rate,
    adjusted_growth_rate, fiscal_year, pricingdate,

       
    round(variance, 2) * 100 as variance,
    round(adjusted_growth_rate, 3) * 100 as adjusted_growth,
    round(adjusted_dcf, 2) as adjusted_dcf,
    
    round((market_dcf - current_price) * 100 / current_price, 2) as market_to_price,
    round((dcf - current_price) * 100 / current_price, 2) as dct_to_price,
    round(market_dcf, 2) as market_dcf, 
    round(dcf, 2) normal_dcf,
    round(current_growth_rate, 3) * 100 as growth_rate,
    round((adjusted_dcf - current_price) * 100 / current_price, 2) as adjusted_dcf_to_price


                from model.dcf_values_current 
                    where total_years = 5
                    and fiscal_year = %s 
                    and is_active = 1
                    and current_price > 0.5
                    and adjusted_growth_rate is not NULL
                    and adjusted_dcf is not NULL
                    and variance is not NULL
                    and current_growth_rate is not NULL
        
        

"""

rv = model_db.fetch_all_rows(year_sql, args=[2019])
print (rv.row_count)

2758


In [27]:
for row in rv.query_data:
    if not isinstance(row.adjusted_dcf_to_price, float):
        print(row.adjusted_dcf_to_price)

In [28]:
for row in rv.query_data:
    
    dto = dcf_mapper.get_dto(symbol=row.symbol, 
        exchangesymbol=row.exchangesymbol,
        tradingitemid=row.tradingitemid,
        price=row.current_price,
        pricing_date=row.pricingdate,
        average_growth_rate=row.current_growth_rate,
        adjusted_growth=row.adjusted_growth_rate,
        dcf=row.normal_dcf,
        market_dcf=row.market_dcf,
        adjusted_dcf=row.adjusted_dcf,
        adjusted_dcf_to_price=row.adjusted_dcf_to_price,
        fiscal_year=row.fiscal_year)

    
    if row.variance >= 65:
        dto.is_yearly_consistent_rate = 1
    else:
        dto.is_yearly_consistent_rate = 0
    next_year_price_data_list = None
    try:
        next_year_price_data_list = get_min_max_current_price(row.symbol, row.pricingdate)
    except:
        print("no daily data for {}".format(row.symbol))
    
    if not next_year_price_data_list:
        print("no daily data for {}".format(row.symbol))
        continue
        
    min_price, max_price, year_end_price, end_pricing_date = next_year_price_data_list
    dto.year_min_price = min_price
    dto.year_max_price = max_price
    dto.year_end_price = year_end_price
    dto.end_pricing_date = end_pricing_date
    
    pred_data = list(row[12:])
    try:
        pred_value = logic_mapper.predict("dcf_adj", pred_data)
    except Exception as error:
        print("predict failed: {}- {}".format(row.symbol, error))
        continue
    
    direction = 0
    
    try:
        up = logic_mapper.predict("dcf_adj_up", pred_data)
        down = logic_mapper.predict("dcf_adj_down", pred_data)
        
        if pred_value > 10 and up == 1:
            direction = 1
        elif pred_value < -10 and down == -1:
            direction = -1
    except Exception as error:
        print (error)
        
        pass
    
    dto.predict_direction = direction
    
    dto.dcf_ai_score = round(pred_value, 1)
  
    dcf_mapper.insert(dto)
    break
    

 data len is not matched ['last_growth_rate', 'last_market_dcf', 'last_dcf', 'dcf_to_price', 'market_dcf_to_price', 'last_variance', 'last_adjust_growth', 'last_adjusted_dcf', 'adjusted_dcf_to_price']


In [ ]:
values = []
table = "model.dcf_values_current"
fields = ["symbol","exchangesymbol","tradingitemid","is_active","current_price","market_growth_rate","current_growth_rate","adjusted_growth_rate","variance","fiscal_year","pricingdate","total_years","market_dcf","dcf","adjusted_dcf""]

This way I have to initialize more attribute to handle try and exeption and orderly append to the value
Then:
Generate_sql = create_django_insert_builder(table_name, field_names, values)
Then:
self._execute_django_dml(Generate_sql)
I have no clue if there is insert successfully or not or updated successfully or not

In [ ]:
self._execute_django_dml(sql)